In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from hmm import HiddenMarkovModel
from hmm_fb import HiddenMarkovModel_FB


tf.logging.set_verbosity(tf.logging.WARN)

%matplotlib inline

In [2]:
def dptable(state_prob):
    print(" ".join(("%8d" % i) for i in range(state_prob.shape[0])))
    for i, prob in enumerate(state_prob.T):
        print("%.7s: " % states[i] +" ".join("%.7s" % ("%f" % p) for p in prob))

In [3]:
p0 = np.array([0.6, 0.4])

emi = np.array([[0.5, 0.1],
                [0.4, 0.3],
                [0.1, 0.6]])

trans = np.array([[0.7, 0.3],
                  [0.4, 0.6]])

states = {0:'Healthy', 1:'Fever'}
obs = {0:'normal', 1:'cold', 2:'dizzy'}

obs_seq = np.array([0, 0, 1, 2, 2])

In [4]:
df_p0 = pd.DataFrame(p0, index=["Healthy", "Fever"], columns=["Prob"])
df_emi = pd.DataFrame(emi, index=["Normal", "Cold", "Dizzy"], columns=["Healthy", "Fever"])
df_trans = pd.DataFrame(trans, index=["fromHealthy", "fromFever"], columns=["toHealthy", "toFever"])

In [5]:
df_p0

,Prob
Healthy,0.6
Fever,0.4


In [6]:
df_trans

,toHealthy,toFever
fromHealthy,0.7,0.3
fromFever,0.4,0.6


In [7]:
df_emi

,Healthy,Fever
Normal,0.5,0.1
Cold,0.4,0.3
Dizzy,0.1,0.6


In [8]:
model =  HiddenMarkovModel(trans, emi, p0)
states_seq, state_prob = model.run_viterbi(obs_seq, summary=True)

print("Observation sequence: ", [obs[o] for o in obs_seq])
print("Most likely States: ",[states[s] for s in states_seq])
df = pd.DataFrame(state_prob.T, index=["Healthy", "Fever"])
df.style.set_properties(**{'text-align': 'left'})

Observation sequence:  ['normal', 'normal', 'cold', 'dizzy', 'dizzy']
Most likely States:  ['Healthy', 'Healthy', 'Healthy', 'Fever', 'Fever']


In [9]:
p0 = np.array([0.5, 0.5])

emi = np.array([[0.9, 0.2],
                [0.1, 0.8]])

trans = np.array([[0.7, 0.3],
                  [0.3, 0.7]])

states = {0:'rain', 1:'no_rain'}
obs = {0:'umbrella', 1:'no_umbrella'}

obs_seq = np.array([1, 1, 0, 0, 0, 1])

In [10]:
model =  HiddenMarkovModel_FB(trans, emi, p0)

results = model.run_forward_backward(obs_seq)
result_list = ["Forward", "Backward", "Posterior"]

for state_prob, path in zip(results, result_list) :
    inferred_states = np.argmax(state_prob, axis=1)
    print()
    print(path)
    dptable(state_prob)
    print()

print("="*60)
print("Most likely Final State: ",states[inferred_states[-1]])
print("="*60)


Forward
       0        1        2        3        4        5        6
rain: 0.50000 0.11111 0.06163 0.68386 0.85819 0.89029 0.19256
no_rain: 0.50000 0.88888 0.93837 0.31613 0.14180 0.10971 0.80743


Backward
       0        1        2        3        4        5        6
rain: 0.32814 0.37709 0.65637 0.64477 0.58110 0.34444 1.00000
no_rain: 0.67186 0.62290 0.34363 0.35522 0.41889 0.65555 1.00000


Posterior
       0        1        2        3        4        5        6
rain: 0.32814 0.07035 0.11146 0.79701 0.89357 0.81002 0.19256
no_rain: 0.67186 0.92965 0.88853 0.20298 0.10643 0.18997 0.80743

Most likely Final State:  no_rain


In [11]:
def generate_HMM_observation(num_obs, pi, T, E):
    
    def drawFrom(probs):
        return np.where(np.random.multinomial(1,probs) == 1)[0][0]

    obs = np.zeros(num_obs)
    states = np.zeros(num_obs)
    
    states[0] = drawFrom(pi)
    obs[0] = drawFrom(E[:, int(states[0])])
    
    for t in range(1,num_obs):
        states[t] = drawFrom(T[int(states[t-1]),:])
        obs[t] = drawFrom(E[:, int(states[t])])
        
    return obs, states

In [12]:
True_pi = np.array([0.5, 0.5])

True_T = np.array([[0.85, 0.15],
                  [0.12, 0.88]])

True_E = np.array([[0.8, 0.0],
                   [0.1, 0.0],
                   [0.1, 1.0]])

In [13]:
obs_seq, states = generate_HMM_observation(50, True_pi, True_T, True_E)
print("First 15 Obersvations:  ", obs_seq[:15])
print("First 15 Hidden States: ", states[:15])

First 15 Obersvations:   [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  0.  0.  0.]
First 15 Hidden States:  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.]


In [14]:
init_pi = np.array([0.5, 0.5])

init_T = np.array([[0.5, 0.5],
                  [0.5, 0.5]])

init_E = np.array([[0.3, 0.2],
                   [0.3, 0.5],
                   [0.4, 0.3]])

In [15]:
model =  HiddenMarkovModel(init_T, init_E, init_pi, epsilon=0.00001, maxStep=20)

trans0, transition, emission, c = model.run_Baum_Welch_EM(obs_seq, summary=False, monitor_state_1=False)

In [17]:
print("Transition Matrix: ")
print(transition)
print()

print("Emission Matrix: ")
print(emission)
print()

print("Reached Convergence: ")
print(c)

Transition Matrix: 
[[ 0.55541573  0.44458427]
 [ 0.59577175  0.40422825]]

Emission Matrix: 
[[ 0.47041126  0.39942016]
 [ 0.03756457  0.08993714]
 [ 0.49202417  0.5106427 ]]

Reached Convergence: 
True
